# Tax Lot Management


Portfolios in LUSID can be created using any of the following tax lot accounting methods:
- Average Cost (default in LUSID)
- First In First Out (FIFO)
- Last In First Out (LIFO)
- Lowest Cost First
- Highest Cost First

These will determine how tax lots are used to update the cost basis of a transaction portfolio when booking various transactions.
Helpful KB articles:
- [What are the supported tax-lot accounting methods in LUSID?](https://support.lusid.com/knowledgebase/article/KA-01886/en-us)
- [How do I handle different tax lot accounting conventions?](https://support.lusid.com/knowledgebase/article/KA-01887/en-us)

## Initial Setup

This section will set up the parameters and methods used in section 2, to compare accounting methods.

In [ ]:
# Import common libraries
import os
import pandas as pd
import logging
import pytz
import json
import random
import dateutil
from datetime import datetime, timezone, timedelta
from IPython.core.display import HTML
from enum import Enum
logging.basicConfig(level = logging.INFO)

# Import LUSID libraries
import lusid as lu
import lusid.models as lm

import lusidjam
import lusid.extensions as le
from finbourne_sdk_utils.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from finbourne_sdk_utils.lpt.lpt import to_date
from finbourne_sdk_utils import cocoon as cocoon

# Set pandas display options
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.options.display.float_format = "{:,.2f}".format

# Authenticate to SDK
# Run the Notebook in Jupyterhub for your LUSID domain and authenticate automatically
secrets_path = os.getenv("FBN_SECRETS_PATH")
if secrets_path is None:
    secrets_path = os.path.join(os.path.dirname(os.getcwd()), "secrets.json")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
config_loaders=[
    le.ArgsConfigurationLoader(access_token = lusidjam.RefreshingToken(), app_name = "LusidJupyterNotebook"),
    le.EnvironmentVariablesConfigurationLoader(),
    le.SecretsFileConfigurationLoader(secrets_path)]
api_factory = le.SyncApiClientFactory(config_loaders=config_loaders)

# Confirm success
api_client = api_factory.build(lu.ApplicationMetadataApi)
api_url = api_client.api_client.configuration._base_path.replace("api","")

print ('LUSID Environment :', api_url + "docs")
display(pd.DataFrame(api_client.get_lusid_versions().to_dict()))

In [ ]:
transaction_portfolios_api = api_factory.build(lu.TransactionPortfoliosApi)
portfolios_api = api_factory.build(lu.PortfoliosApi)
derived_portfolios_api = api_factory.build(lu.DerivedTransactionPortfoliosApi)
property_definitions_api = api_factory.build(lu.PropertyDefinitionsApi)
instruments_api = api_factory.build(lu.InstrumentsApi)
corporate_action_sources_api = api_factory.build(lu.CorporateActionSourcesApi)
quotes_api = api_factory.build(lu.QuotesApi)
recipes_api = api_factory.build(lu.ConfigurationRecipeApi)
aggregation_api = api_factory.build(lu.AggregationApi)

### Variables

In [ ]:
scope = "taxlot_management_example"
portfolio_code_avg = "tax-lot-avgcost"
portfolio_code_fifo = "tax-lot-fifo"
portfolio_code_lifo = "tax-lot-lifo"
portfolio_code_high = "tax-lot-highestcost"
portfolio_code_low = "tax-lot-lowestcost"
portfolio_created_date = portfolio_creation_date = datetime(2010, 1, 1, tzinfo=timezone.utc)

ca_code = "tax_lot_stock_split"

recipe_code = "valuation"

property_code = "sleeve"

### Data Clean Up

In [ ]:
# Delete Corporate Action Source & Action
try:
    resp = corporate_action_sources_api.delete_corporate_action_source(
        scope=scope,
        code=ca_code
    )
    if resp:
        print(f"DELETED: Corporate Actions Source '{scope}/{ca_code}'")
except lu.ApiException as e:
    print(json.loads(e.body)["title"])

# Delete Configuration Recipe
try:
    resp = recipes_api.delete_configuration_recipe(scope=scope, code=recipe_code)
    if resp:
        print(f"DELETED: Configuration Recipe '{scope}/{recipe_code}'")
except lu.ApiException as e:
    print(json.loads(e.body)["title"])

# Delete Properties
try:
    resp = property_definitions_api.delete_property_definition(
        domain="Transaction",
        scope=scope,
        code=property_code
    )
    if resp:
        print(f"DELETED: Property 'Transaction/{scope}/sleeve'")
except lu.ApiException as e:
    print(json.loads(e.body)["title"])

# Delete Portfolios
for code in [portfolio_code_fifo, portfolio_code_lifo, portfolio_code_high, portfolio_code_low, portfolio_code_avg]:
    try:
        resp = portfolios_api.delete_portfolio(
            scope=scope,
            code=code
        )
        if resp:
            print(f"DELETED: Portfolio '{code}'")
    except lu.ApiException as e:
        print(json.loads(e.body)["title"])

        
# Delete Quotes
quotes = quotes_api.list_quotes_for_scope(scope)
if len(quotes.values) > 0:
    quotes_api.delete_quotes(
        scope, 
        request_body={
            f"{quote.quote_id.quote_series_id.instrument_id}_{quote.as_at}": quote.quote_id
            for quote in quotes.values
        }
    )
    print (f"DELETED: {len(quotes.values)} quotes")

## Loading our Data

In [ ]:
txns = pd.read_csv(
    "data/taxlot-accounting/transactions.csv",
    parse_dates=['transaction_date','settlement_date'], 
    date_format='%Y-%m-%d')

txns.head()

### Create Portfolio

When creating a transaction portfolio in LUSID, we pass in a parameter of the `accounting_method` to determine how the tax lots are calculated. If not specified, **Average Cost** is the default method used in LUSID.

In [ ]:
# Create corporate action source, this is covered further in section 4
try:
    source_request = lm.CreateCorporateActionSourceRequest(
        scope=scope,
        code=ca_code,
        display_name="Tax Lot Corporate Action Source",
        description="Corporate Actions source for tax lot sample notebook",
    )

    corporate_action_sources_api.create_corporate_action_source(
        create_corporate_action_source_request=source_request
    )

except lu.ApiException as e:
    print(json.loads(e.body)["title"])

In [ ]:
# Create sleeve property for sub-holding key
try:
    resp = property_definitions_api.create_property_definition(
        create_property_definition_request=lm.CreatePropertyDefinitionRequest(
            domain="Transaction",
            scope=scope,
            code=property_code,
            value_required=None,
            display_name="Sleeve",
            data_type_id=lm.ResourceId(scope="system", code="string"),
            life_time=None,
        )
    )
    print(f"{resp.key} property created")
    
except lu.ApiException as e:
    if json.loads(e.body)["code"] == 124: # PropertyAlreadyExists
        print(json.loads(e.body)["title"])
    else:
        raise e

In [ ]:
def create_portfolio(name, accounting_method):
    try:
        # Create request body
        portfolio_request = lm.CreateTransactionPortfolioRequest(
            display_name=f"Tax Lot Management Example - {accounting_method}",
            code=name,
            base_currency="GBP",
            corporate_action_source_id=lm.ResourceId(scope=scope, code=ca_code),
            accounting_method=accounting_method, # If not specified - AverageCost is the default accounting method.
            created=portfolio_created_date,
            sub_holding_keys=[f"Transaction/{scope}/sleeve"],
        )

        # Upload new portfolio to LUSID
        response = transaction_portfolios_api.create_portfolio(
            scope=scope, create_transaction_portfolio_request=portfolio_request
        )

        created = response.version.effective_from
        print(
            f"Portfolio '{response.id.code}', in scope {scope} created effective from: "
            f"{created.year}/"
            f"{created.month}/"
            f"{created.day}")
        
    except lu.ApiException as e:
        print(json.loads(e.body)["title"])

In [ ]:
def create_derived_portfolio(portfolio_code, parent_code, accounting_method, ca_code):
    try:
        derived_portfolio_request = lm.CreateDerivedTransactionPortfolioRequest(
            display_name=f"Tax Lot Management Example - {accounting_method}",
            code=portfolio_code,
            parent_portfolio_id=lm.ResourceId(scope=scope, code= parent_code),
            created=portfolio_created_date,
            corporate_action_source_id=lm.ResourceId(scope=scope, code=ca_code),
            accounting_method=accounting_method,
            sub_holding_keys=[f"Transaction/{scope}/sleeve"],
        )
        response = derived_portfolios_api.create_derived_portfolio(
            scope=scope,
            create_derived_transaction_portfolio_request = derived_portfolio_request)
        created = response.version.effective_from
        print(
        f"Derived portfolio '{response.id.code}', in scope {scope} created effective from: "
        f"{created.year}/"
        f"{created.month}/"
        f"{created.day}")

    except lu.ApiException as e:
        print(json.loads(e.body)["title"])

In [ ]:
# Create a parent portfolio.
create_portfolio(portfolio_code_avg, "AverageCost")

In [ ]:
# Create portfolios derived from the AverageCost portfolio. (Tax lot type of the parent portfolio does not affect the derived
# portfolios.)
create_derived_portfolio(portfolio_code_fifo, portfolio_code_avg, "FirstInFirstOut", ca_code)

create_derived_portfolio(portfolio_code_lifo, portfolio_code_avg, "LastInFirstOut", ca_code)

create_derived_portfolio(portfolio_code_high, portfolio_code_avg, "HighestCostFirst", ca_code)

create_derived_portfolio(portfolio_code_low, portfolio_code_avg, "LowestCostFirst", ca_code)

### Create Instruments

In [ ]:
# Create instruments
batch_upsert_request = {}
uploaded = []

for index, row in txns.iterrows():
    client_internal = row['client_internal']
    ticker = row['ticker']
    name = row['name']
    
    if client_internal.startswith("CASH") or client_internal in uploaded:
        continue
    else:
        batch_upsert_request[client_internal] = lm.InstrumentDefinition(
            name=name,
            identifiers={ 
                "ClientInternal": lm.InstrumentIdValue(value=client_internal),
                "Ticker": lm.InstrumentIdValue(value=ticker)
            },
            definition=lm.Equity(
                instrument_type="Equity",
                dom_ccy="USD"
            )
        )
        uploaded.append(client_internal)
        

# Upsert new instruments to LUSID
instrument_response = instruments_api.upsert_instruments(
    request_body=batch_upsert_request
)

# Check response was successful
if len(instrument_response.failed) > 0:
    raise AssertionError("Instruments upsert failed. Inspect response for more detail")
    
print(f"{len(instrument_response.values)} instruments uploaded")

### Upload Transactions

In [ ]:
def upload_transactions_to_portfolio(portfolio_name, transactions):
    transactions_request = []

    for index, row in transactions.iterrows():

        if row["client_internal"].startswith("CASH"):
            instrument_identifier = {"Instrument/default/Currency": "USD"}

        else:
            instrument_identifier = {
                    "Instrument/default/ClientInternal": row["client_internal"]
                }

        # Build request body
        transactions_request.append(
            lm.TransactionRequest(
                transaction_id=row["transaction_id"],
                type=row["type"],
                instrument_identifiers=instrument_identifier,
                transaction_date=row["transaction_date"].isoformat() + "Z",
                settlement_date=row["settlement_date"].isoformat() + "Z",
                units=row["units"],
                transaction_price=lm.TransactionPrice(price=row["transaction_price"], type="Price"),
                total_consideration=lm.CurrencyAndAmount(
                    amount=row["total_consideration"], currency="USD"
                ),
                properties={
                    f"Transaction/{scope}/{property_code}": 
                    lm.PerpetualProperty(
                        key=f"Transaction/{scope}/{property_code}",
                        value=lm.PropertyValue(label_value=row['sleeve'])
                    ),
                    f"Transaction/default/TradeToPortfolioRate": 
                    lm.PerpetualProperty(
                        key=f"Transaction/default/TradeToPortfolioRate",
                        value=lm.PropertyValue(
                            metric_value=lm.MetricValue(value=row["trade_to_portfolio_rate"])
                        )
                    )
                }
            )
        )

    # Make upsert transactions call to LUSID
    txn_response = transaction_portfolios_api.upsert_transactions(
            scope=scope, code=portfolio_code_avg, transaction_request=transactions_request
    )

    return txn_response.version.as_at_date


In [ ]:
transactions_upsert_as_at = upload_transactions_to_portfolio(portfolio_code_avg, txns)

print(f"{len(txns)} transactions upserted to portfolio: {portfolio_code_avg}")

## Cost Basis Comparison

Holdings before our sell transaction show the three distinct tax lots and corresponding transactions. We can see that each tax lot has a distinct cost basis based on the transaction price and total consideration of each transaction.

When calling `get_holdings()` in the `display_holding_positions_by_taxlot` method, setting the flag `by_taxlots=true` returns the holdings separated by tax lots, as shown below.

In [ ]:
# Prints quick summary from a get_holdings() response of positions for a given effective_at date broken down by tax lots
def display_holding_positions_by_taxlot(effective_at, portfolio_code, show_taxlots = True, as_at = None):
    # Get holdings
    response = transaction_portfolios_api.get_holdings(
        scope=scope,
        code=portfolio_code,
        effective_at = effective_at.isoformat(),
        as_at = as_at,
        property_keys=["Instrument/default/Name"],
        by_taxlots=show_taxlots
    )

    # Inspect holdings response for the given effective_at day
    hld = [i for i in response.values]

    names = []
    cost = []
    units = []
    txnid = []
    pchprice = []
    pchdate = []

    for item in hld:
        if item.holding_type_name == "Position":
            names.append(item.properties["Instrument/default/Name"].value.label_value)
            cost.append(item.cost.amount)
            units.append(item.units)
            if show_taxlots:
                txnid.append(item.properties["Holding/default/TaxlotId"].value.label_value)
                pchprice.append(item.properties["Holding/default/TaxlotPurchasePrice"].value.metric_value.value)
                pchdate.append(item.properties["Holding/default/TaxlotPurchaseDate"].value.label_value.replace("T00:00:00.0000000+00:00",""))
                
    data = {"cost_basis": cost, "units": units}
    if show_taxlots:
        data = {"transaction_id": txnid, **data , "purchase_price":pchprice, "purchase_date":pchdate}
    return pd.DataFrame(data=data, index=names)

# Displays a link to view portfolio in LUSID
def print_url_to_holdings(effective_at, portfolio_code):
    date = effective_at.strftime('%Y-%m-%d')
    display(HTML(f'<a href="{api_url}app/dashboard/holdings?scope={scope}&code={portfolio_code}&entityType=Portfolio&taxLots=true&effectiveDate={date}" target="_blank">See holdings positions by tax lot in LUSID</a>'))

In [ ]:
display_holding_positions_by_taxlot(datetime(year=2020, month=3, day=7, tzinfo=pytz.UTC), portfolio_code_fifo)

In [ ]:
print_url_to_holdings(datetime(year=2020, month=3, day=7, tzinfo=pytz.UTC), portfolio_code_fifo)

We'll now look at the final positions of our portfolios under the 5 accounting methods described above. The units sold for each quity can be seen in the 'units' column.

### Average Cost
Average Cost uses the average price of the final holdings of our portfolios. In the case of **Apple**, for example, this is $ \frac{49,603,000}{300,000} = 165.34.$
When using the **Average Cost** method, our cost basis is averaged across all transactions. Splitting out our holdings by tax lot is thus not applicable. The total cost basis after the sale will be calculated as $165.34 \times 150,000 = 24,801,500$.
**Average Cost** is the default accounting method for a portfolio created in LUSID.

In [ ]:
display_holding_positions_by_taxlot(datetime(year=2020, month=3, day=14, tzinfo=pytz.UTC), portfolio_code_avg, show_taxlots=False)

In [ ]:
print_url_to_holdings(datetime(year=2020, month=3, day=7, tzinfo=pytz.UTC), portfolio_code_avg)

### First In First Out (FIFO)
For **Apple**, using FIFO, our first tax lot (all 100,000 units coming from `txn_002`) will be fully sold, while half of our second tax lot (50,000 units coming from `txn_003`) will be sold. Below we show the remaining holdings with 50,000 units bought in `txn_003` and all the units bought in `txn_004` in separate tax lots.

In [ ]:
display_holding_positions_by_taxlot(datetime(year=2020, month=3, day=14, tzinfo=pytz.UTC), portfolio_code_fifo)

In [ ]:
print_url_to_holdings(datetime(year=2020, month=3, day=7, tzinfo=pytz.UTC), portfolio_code_fifo)

### Last In Last Out (LIFO)
For **Apple**, using LIFO, our first tax lot (all 100,000 units coming from `txn_004`) will be fully sold, while half of our second tax lot (50,000 units coming from `txn_003`) will be sold.
The difference in the total cost basis compared to the **FIFO** method, is due to the difference in total consideration between `txn_002` and `txn_004`.

In [ ]:
display_holding_positions_by_taxlot(datetime(year=2020, month=3, day=14, tzinfo=pytz.UTC), portfolio_code_lifo)

In [ ]:
print_url_to_holdings(datetime(year=2020, month=3, day=7, tzinfo=pytz.UTC), portfolio_code_lifo)

### Highest Cost First

Using Highest Cost First, the equities with the highest transaction price will be sold first. Take **Apple** for example, in the sell transaction of 150,000 units, all units from `txn_002` and 50,000 units of `txn_004` will be sold. All units from `txn_003` and 50,000 units from `txn_004` remain in the cost basis as shown in the display below.

In [ ]:
display_holding_positions_by_taxlot(datetime(year=2020, month=3, day=14, tzinfo=pytz.UTC), portfolio_code_high)

In [ ]:
print_url_to_holdings(datetime(year=2020, month=3, day=7, tzinfo=pytz.UTC), portfolio_code_high)

### Lowest Cost First

Using Lowest Cost First, the equities with the lowest transaction price will be sold first. For **Apple**, in the sell transaction of 150,000 units, all units from `txn_003` and 50,000 units of `txn_004`. All units from `txn_002` and 50,000 units from `txn_004` remain in the cost basis as shown in the display below.
The difference in the cost basis compared to the **Highest First** method, is due to the difference in total consideration between `txn_002` and `txn_004`.

In [ ]:
display_holding_positions_by_taxlot(datetime(year=2020, month=3, day=14, tzinfo=pytz.UTC), portfolio_code_low)

In [ ]:
print_url_to_holdings(datetime(year=2020, month=3, day=7, tzinfo=pytz.UTC), portfolio_code_low)

## Corporate Actions

Below, we apply a 2 for 1 stock split corporate action on Microsoft shares. This has the effect of doubling the units of an equity that we hold.

In [ ]:
# Set conditions of Corporate Action
transitions = [
    lm.CorporateActionTransitionRequest(
        input_transition=lm.CorporateActionTransitionComponentRequest(
            #instrumentScope = "d",
            instrument_identifiers={
                "Instrument/default/ClientInternal": "EQUITY_MSFT"
            },
            units_factor=1,
            cost_factor=1
        ),
        output_transitions=[
            lm.CorporateActionTransitionComponentRequest(
             #   instrument_scope = "f",
                instrument_identifiers={
                    "Instrument/default/ClientInternal": "EQUITY_MSFT"
                },
                units_factor=2,
                cost_factor=1,
            )
        ],
    )
]

# Create Corporate Action
split_request = lm.UpsertCorporateActionRequest(
    corporate_action_code="SS001",
    announcement_date=datetime(year=2020, month=3, day=5, tzinfo=pytz.UTC),
    ex_date=datetime(year=2020, month=3, day=6, tzinfo=pytz.UTC),
    record_date=datetime(year=2020, month=3, day=6, tzinfo=pytz.UTC),
    payment_date=datetime(year=2020, month=3, day=6, tzinfo=pytz.UTC),
    transitions=transitions,
)

result = corporate_action_sources_api.batch_upsert_corporate_actions(
    scope=scope, code=ca_code, upsert_corporate_action_request=[split_request]
)

Before the stock split.

In [ ]:
display_holding_positions_by_taxlot(datetime(year=2020, month=3, day=5, tzinfo=pytz.UTC), portfolio_code_low).query("index == 'Microsoft'")

After the stock split. Note how the units have doubled for Microsoft.

In [ ]:
display_holding_positions_by_taxlot(datetime(year=2020, month=3, day=7, tzinfo=pytz.UTC), portfolio_code_low).query("index == 'Microsoft'")

We'll also add a corporate action for a dividend from Apple which we'll use later when exploring backdating.

In [ ]:
# Set conditions of Corporate Action
transitions = [
    lm.CorporateActionTransitionRequest(
        input_transition=lm.CorporateActionTransitionComponentRequest(
            instrument_identifiers={
                "Instrument/default/ClientInternal": "EQUITY_APPL"
            },
            units_factor=1,
            cost_factor=1
        ),
        output_transitions=[
            lm.CorporateActionTransitionComponentRequest(
                instrument_identifiers={
                    "Instrument/default/Currency": "USD"
                },
                units_factor=0.82,
                cost_factor=1
            )
        ],
    )
]

# Create Corporate Action
dividend_request = lm.UpsertCorporateActionRequest(
    corporate_action_code="SS002",
    announcement_date=datetime(year=2020, month=4, day=30, tzinfo=pytz.UTC),
    ex_date=datetime(year=2020, month=5, day=11, tzinfo=pytz.UTC),
    record_date=datetime(year=2020, month=5, day=11, tzinfo=pytz.UTC),
    payment_date=datetime(year=2020, month=5, day=14, tzinfo=pytz.UTC),
    transitions=transitions,
)

result = corporate_action_sources_api.batch_upsert_corporate_actions(
    scope=scope, code=ca_code, upsert_corporate_action_request=[dividend_request]
)

## Demo Walkthrough

This sections details some useful links and data that helps to illustrate the taxlot offering we can provide.

Key Dates:
- **04/03/2020** : Cash paid in and first transaction for MSFT only
- **05/03/2020** : Cash settles and majority of equities get first transaction
- **06/03/2020** : Corporate action stock split (2 for 1) for Microsoft
- **07/03/2020** : Final Buy transactions executed
- **08/03/2020** : All sell transactions executed
- **10/03/2020** : All Buy transactions settled
- **11/03/2020** : All Sell transactions settled

### Initialise Functions

Declare the functions that we will use throughout this demo. We have done this at the top of this section to help with readibility.

In [ ]:
# Demo class for below function
class Demo(Enum):
    HOLDINGS = "holdings"
    TRANSACTIONS = "transactions"
    CORPORATE_ACTIONS = "actions"
    VALUATION = "valuations"
    
# Generate links for each demo type
def generate_href(demo_type, effective_at, portfolio_list=None, as_at=None):
    # Create URLs
    if demo_type not in [demo_type.HOLDINGS, demo_type.VALUATION]:
        from_date = (effective_at - timedelta(60)).strftime('%Y-%m-%d')
        to_date = effective_at.strftime('%Y-%m-%d')
        # Create link for Corporate Actions
        if demo_type.name == "CORPORATE_ACTIONS":
            display(HTML(f'''<b>Corporate Actions:</b> <a href="{api_url}app/data-management/actions?actionsDateFrom={from_date}&actionsDateTo={to_date}&corporateSourceScope={scope}&corporateSourceCode={ca_code}&corporateSourceName=Tax Lot Corporate Action Source" target="_blank">See corporate actions in LUSID</a>'''))
            return
        # Format date for Transactions
        date_string = f"dateFrom={from_date}&dateTo={to_date}"
    else:
        # Format date for Holdings
        date_string = f"effectiveDate={effective_at.strftime('%Y-%m-%d')}"

    if as_at is not None:
        as_at = as_at + timedelta(milliseconds=1)  # manually round up as web can only handle milliseconds
        date_string += f"&asAt={as_at.isoformat(timespec='milliseconds').replace('+00:00', 'Z')}"
    
    if demo_type == Demo.VALUATION:
        date_string += f"&recipeScope={scope}&recipeCode={recipe_code}&"
        
    # Create links for Holdings and Transactions        
    if portfolio_list:
        for portfolio_code in portfolio_list:
            display(HTML(f'''<b>{portfolio_code[0]}:</b> <a href="{api_url}app/dashboard/{demo_type.value}?scope={scope}&code={portfolio_code[1]}&entityType=Portfolio&taxLots=true&{date_string}" target="_blank">See {demo_type.value} by tax lot in LUSID</a>'''))
    else:
        raise Exception(f"Parameter 'portfolio_list' cannot be None for {demo_type}")

# Get the gain and loss data
def get_gl(val, pricing=False):
    if not pricing:
        return val.realised_gain_loss[0]
    else:
        sell_units = val.realised_gain_loss[0].units
        sell_amount = val.realised_gain_loss[0].cost_portfolio_ccy.amount
        return sell_amount/sell_units

# Get name of instrument
def get_name(val):
    if 'Instrument/default/ClientInternal' in val.instrument_identifiers:
        return val.instrument_identifiers['Instrument/default/ClientInternal']
    elif 'Instrument/default/Currency' in val.instrument_identifiers:
        return val.instrument_identifiers['Instrument/default/Currency']

# Get gain & loss data by building transactions
def build_transactions(effective_at, portfolio_code, as_at=None):
    from_date = (effective_at - timedelta(60)).strftime('%Y-%m-%d')
    to_date = effective_at.strftime('%Y-%m-%d')
    sleeve_key = f'Transaction/{scope}/{property_code}'

    result = transaction_portfolios_api.build_transactions(
        scope=scope,
        code=portfolio_code,
        as_at=as_at,
        transaction_query_parameters=lm.TransactionQueryParameters(
            start_date=from_date,
            end_date=to_date
        )
    )

    vals = result.values # for readibility
    gain_loss_data = {
        "Name": [get_name(val) for val in vals],
        "Type": [val.type for val in vals],
        "TxnId": [val.transaction_id for val in vals],
        "Transaction Date": [val.transaction_date.date() for val in vals],
        "Units": [val.units for val in vals],
        "Cost Holding Ccy (USD)": [sum(gl.cost_trade_ccy.amount for gl in val.realised_gain_loss) if val.realised_gain_loss else None for val in vals],
        "Realised Trade Ccy (USD)": [sum(gl.realised_trade_ccy.amount for gl in val.realised_gain_loss) if val.realised_gain_loss else None for val in vals],
        "Cost Portfolio Ccy (GBP)": [sum(gl.cost_portfolio_ccy.amount for gl in val.realised_gain_loss) if val.realised_gain_loss else None for val in vals],
        "Realised Trade Ccy - Market Component (GBP)": [sum(gl.realised_market.amount for gl in val.realised_gain_loss) if val.realised_gain_loss else None for val in vals],
        "Realised Trade Ccy - Currency Component (GBP)": [sum(gl.realised_currency.amount for gl in val.realised_gain_loss) if val.realised_gain_loss else None for val in vals],
        "Realised Trade Ccy - Total (GBP)": [sum(gl.realised_total.amount for gl in val.realised_gain_loss) if val.realised_gain_loss else None for val in vals],
    }
    return pd.DataFrame(data=gain_loss_data).sort_values(by=['Name', 'Transaction Date'])

In [ ]:
portfolio_list = [
    ["Average Cost", portfolio_code_avg],
    ["FIFO", portfolio_code_fifo],
    ["LIFO", portfolio_code_lifo],
    ["Highest First", portfolio_code_high],
    ["Lowest First", portfolio_code_low]
]

### Show Holdings

Holdings links (with taxlots on). Try grouping by instrument and then by sleeve.

In [ ]:
generate_href(Demo.HOLDINGS, datetime(year=2020, month=3, day=11, tzinfo=pytz.UTC), portfolio_list,
             as_at=transactions_upsert_as_at)

### Show Transactions

We recommend filtering for a single equity (eg. Pepsico) and showing how its transactions vary depending on the tax lot.

In [ ]:
generate_href(Demo.TRANSACTIONS, datetime(year=2020, month=3, day=11, tzinfo=pytz.UTC), portfolio_list,
             as_at=transactions_upsert_as_at)                                

### Corporate Actions


First, view the corporate actions in the corporate actions screen. You should see the Microsoft equity being impacted.

In [ ]:
generate_href(Demo.CORPORATE_ACTIONS, datetime(year=2020, month=3, day=11, tzinfo=pytz.UTC),
             as_at=transactions_upsert_as_at) 

Next, view the corporate actions within the transaction screen. To find them, filter your transactions for Microsoft and look for an "Adjustment Increase" transaction type. See how the values vary between the average tax lot and the rest.

In [ ]:
generate_href(Demo.TRANSACTIONS, datetime(year=2020, month=3, day=11, tzinfo=pytz.UTC), portfolio_list,
             as_at=transactions_upsert_as_at)                                

### Gain and Loss

Below, we get the gain and loss of each strategy for a single equity, and present it in a DataFrame. You can adjust which equity is being compared by modifying the `conditions` variable

In [ ]:
# change condition to view other equities eg. EQUITY_MSFT etc.
conditions = 'Name == "EQUITY_MSFT" and Type == "Sell"'

effective_at = datetime(year=2020, month=3, day=14, tzinfo=pytz.UTC)

avg_df = build_transactions(effective_at, portfolio_code_avg).query(conditions)
fifo_df = build_transactions(effective_at, portfolio_code_fifo).query(conditions)
lifo_df = build_transactions(effective_at, portfolio_code_lifo).query(conditions)
high_df = build_transactions(effective_at, portfolio_code_high).query(conditions)
low_df = build_transactions(effective_at, portfolio_code_low).query(conditions)

display("Average tax lot", avg_df)
display("FIFO tax lot", fifo_df)
display("LIFO tax lot", lifo_df)
display("Highest tax lot", high_df)
display("Lowest tax lot", low_df)


You can also view the gain and loss by accessing the transactions screen and clicking on the +/- icon on the sell transactions. Use the links below.

In [ ]:
generate_href(Demo.TRANSACTIONS, datetime(year=2020, month=3, day=11, tzinfo=pytz.UTC), portfolio_list,
             as_at=transactions_upsert_as_at)                                

### Backdating

We can also explore the effects of backdating activity under each of our accounting methods.

For example for let us consider that for our holding of Apple:

**1)** We have missed a sale transaction which we need to backdate into the Portfolio

**2)** We need to correct the price on the purchase under txn_004 from `$165.42` to `$164.86`

Let's use our First in First Out (FIFO) Portfolio as an example to understand how this backdated activity impacts our tax lots. 

In [ ]:
# Change this to the accounting method to explore the effect of backdating under
accounting_method = portfolio_code_fifo 

#### Closed Book Positions

Let's start with our closed book positions on the 14th March 2020 before any backdating has occurred. We'll look at our current holdings and our gain and loss.

In [ ]:
effective_at = datetime(year=2020, month=3, day=14, tzinfo=pytz.UTC)

display_holding_positions_by_taxlot(
    effective_at,
    accounting_method,
    as_at=transactions_upsert_as_at
)

In [ ]:
pl_df = build_transactions(
    effective_at,
    accounting_method, 
    as_at=transactions_upsert_as_at).query('Name == "EQUITY_APPL" and Type == "Sell"')

display(accounting_method, pl_df)

In [ ]:
generate_href(
    Demo.HOLDINGS, 
    effective_at, 
    [[accounting_method, accounting_method]],
    as_at=transactions_upsert_as_at
)

generate_href(
    Demo.TRANSACTIONS, 
    effective_at, 
    [[accounting_method, accounting_method]],
    as_at=transactions_upsert_as_at
)

#### Backdated Activity

In [ ]:
backdated_txns = pd.read_csv("data/taxlot-accounting/backdated_transactions.csv", parse_dates=[
    "transaction_date", "settlement_date"
])

#### Backdated Sale

In [ ]:
backdated_transaction = backdated_txns.iloc[0:1]
backdated_transaction.head()

In [ ]:
backdated_sale_as_at = upload_transactions_to_portfolio(portfolio_code_avg, backdated_transaction)

With the backdated sale of Apple and using FIFO, our first tax lot (all 100,000 units coming from txn_002) will continue to be fully sold, however instead of half of our second tax lot (50,000 units coming from txn_003) being sold, the entire tax lot has now been sold, half from our original sale, and half from our backdated sale.

Below we show the remaining holdings with only half the units bought in txn_004 remaining. 

In [ ]:
display_holding_positions_by_taxlot(
    effective_at, 
    accounting_method,
    as_at=backdated_sale_as_at)

Instead of one, we have two sales with their own realised gain/loss. Note that the sale on the 8th of March 2020 has had an increase in its realised gain.

This is because the backdated sale slotted in before the sale on the 8th of March and therefore under FIFO sold the earlier tax lots with a higher puchase price. This left tax lots with a lower purchase price to be sold by the sale on the 8th of March, leading to an increase in the profit on this sale.

In [ ]:
pl_df = build_transactions(
    effective_at, 
    accounting_method,
    as_at=backdated_sale_as_at
).query('Name == "EQUITY_APPL" and Type == "Sell"')

display(accounting_method, pl_df)

In [ ]:
generate_href(
    Demo.HOLDINGS, 
    effective_at, 
    [[accounting_method, accounting_method]],
    as_at=backdated_sale_as_at
)

generate_href(
    Demo.TRANSACTIONS, 
    effective_at, 
    [[accounting_method, accounting_method]],
    as_at=backdated_sale_as_at
)

#### Effect on Corporate Actions

We have a corporate action linked to the Portfolio for a `$0.82` dividend in Apple which is paid on the 14 May 2020 with a record date of the 11th of May 2020.

We can see that the backdated sale automatically updates the effect of the coporate action buy reducing the number of units of Apple we hold on the record date from 150,000 to 50,000. 

This means we get a dividend of `150,000 * $0.82 = $123,000.00` before the backdated sale and `50,000 * $0.82 = 41,000.00` after the backdated sale.

In [ ]:
after_apple_dividend_paid = datetime(year=2020, month=5, day=15, tzinfo=pytz.UTC)

In [ ]:
pl_df = build_transactions(
    after_apple_dividend_paid,
    accounting_method,
    as_at=backdated_sale_as_at - timedelta(milliseconds=1) # Before the backdated sale was added to LUSID
)

display(accounting_method, pl_df)

In [ ]:
pl_df = build_transactions(
    after_apple_dividend_paid,
    accounting_method,
    as_at=backdated_sale_as_at 
)

display(accounting_method, pl_df)

In [ ]:
generate_href(
    Demo.TRANSACTIONS, 
    after_apple_dividend_paid,
    [[accounting_method + " - before backdated sale", accounting_method]],
    as_at=backdated_sale_as_at - timedelta(milliseconds=1)
)

generate_href(
    Demo.TRANSACTIONS, 
    after_apple_dividend_paid,
    [[accounting_method + " - after backdated sale", accounting_method]],
    as_at=backdated_sale_as_at
)

#### Backdated Price Correction

In [ ]:
backdated_price_correction_txn = backdated_txns.iloc[1:2]
backdated_price_correction_txn.head()

In [ ]:
backdated_price_correction_as_at = upload_transactions_to_portfolio(
    portfolio_code_avg, 
    backdated_price_correction_txn
)

With the backdated correction of the purhase price of Apple in txn_004, we can see that our cost basis for Apple made up of the remaining half of the txn_004 tax lot has been reduced from `$8,271,000.00` to `$8,243,000.00` to reflect the corrected price. 

This is a change of (`$8,271,000.00` - `$8,243,000.00`) / `50,000` = `$0.56` per share which is equal to our change in purchase price of `$165.42` - `$164.86` = `$0.56`.

In [ ]:
display_holding_positions_by_taxlot(
    effective_at, 
    accounting_method,
    as_at=backdated_price_correction_as_at
)

With the backdated price correction on txn_004 we can see that our realised gain has increased for our sale on the 8th of March 2023. This is because the cost price has gone down leading to a greater profit for the sale out of this tax lot.

The backdated sale is unaffected as it sold out of earlier tax lots which are unaffected by the price change.

In [ ]:
pl_df = build_transactions(
    effective_at,
    accounting_method,
    as_at=backdated_price_correction_as_at
).query('Name == "EQUITY_APPL" and Type == "Sell"')

display(accounting_method, pl_df)

In [ ]:
generate_href(
    Demo.HOLDINGS, 
    effective_at, 
    [[accounting_method, accounting_method]],
    as_at=backdated_sale_as_at)

generate_href(
    Demo.TRANSACTIONS, 
    effective_at, 
    [[accounting_method, accounting_method]],
    as_at=backdated_price_correction_as_at
)

#### Valuation

We may also want to value our tax lots. To do this we load prices for each of our holdings into LUSID and and construct a recipe to produce a valuation.

Firstly, we construct a recipe which tells LUSID how to find the quotes to value our Portfolio.

In [ ]:
response = recipes_api.upsert_configuration_recipe(
    lm.UpsertRecipeRequest(
        configuration_recipe=lm.ConfigurationRecipe(
            scope=scope,
            code=recipe_code,
            market=lm.MarketContext(
                market_rules=[
                    lm.MarketDataKeyRule(
                        key="Quote.ClientInternal.*",
                        supplier="Lusid",
                        data_scope=scope,
                        quote_type="Price",
                        field="Close"
                    ),
                    lm.MarketDataKeyRule(
                        key="Fx.CurrencyPair.*",
                        supplier="Lusid",
                        data_scope=scope,
                        quote_type="Rate",
                        field="Close",
                        quote_interval="10Y.0D"
                    ),
                ],
                options=lm.MarketOptions(
                    default_scope=scope,
                    attempt_to_infer_missing_fx=True
                )
            ),
        )
    )
)

# Recipe option "holding/taxLotLevelHoldings" defaults to True

print(f"Configuration Recipe '{scope}/{recipe_code}' created")

Then we source and load our quotes.

In [ ]:
prices = pd.read_csv("data/taxlot-accounting/prices.csv")
prices

In [ ]:
quotes_request = {}

# Iterate over the quotes
for index, quote in prices.iterrows():

    quotes_request[str(index)] = lm.UpsertQuoteRequest(
        quote_id=lm.QuoteId(
            quote_series_id=lm.QuoteSeriesId(
                provider='Lusid',
                instrument_id=quote['Identifier'],
                instrument_id_type=quote['IdentifierType'],
                quote_type=quote['Type'],
                field='Close',
            ),
            effective_at=datetime.strptime(quote['Date'], '%Y-%m-%d').replace(tzinfo=timezone.utc).isoformat(),
        ),
        metric_value=lm.MetricValue(
            value=quote['Price'],
            unit=quote['Currency'],
        ),
        lineage="InternalSystem",
    )
    
# Upsert the quotes into LUSID
response = quotes_api.upsert_quotes(scope=scope, request_body=quotes_request)

response.failed

In [ ]:
valuation_request = lm.ValuationRequest(
    recipe_id=lm.ResourceId(scope=scope, code=recipe_code),
    metrics=[
        lm.AggregateSpec(key="Holding/HoldingType", op="Value"),
        lm.AggregateSpec(key="Instrument/default/LusidInstrumentId", op="Value"),
        lm.AggregateSpec(key="Instrument/CoreData/Name", op="Value"),
        lm.AggregateSpec(key="Holding/DomCcy", op="Value"),
        lm.AggregateSpec(key="Holding/Units", op="Value"),
        lm.AggregateSpec(key="Holding/Cost/Dom", op="Value"),
        lm.AggregateSpec(key="Holding/Cost/Pfolio", op="Value"),
        lm.AggregateSpec(key="Holding/default/TaxlotPurchaseDate", op="Value"),
        lm.AggregateSpec(key="Holding/default/TaxlotPurchasePrice", op="Value"),
        lm.AggregateSpec(key="Quotes/Price", op="Value"),
        lm.AggregateSpec(key="Valuation/PV", op="Value"),
        lm.AggregateSpec(key="Valuation/PvInPortfolioCcy", op="Value"),
        lm.AggregateSpec(key="Aggregation/Errors", op="Value"),
        lm.AggregateSpec(key="Holding/HoldingId", op="Value"),
        lm.AggregateSpec(key="Holding/TaxLotId", op="Value"),
        lm.AggregateSpec(key=f"Transaction/{scope}/{property_code}", op="Value"),
    ],
    portfolio_entity_ids=[
        lm.PortfolioEntityId(scope=scope, code=portfolio_code_fifo)
    ],
    valuation_schedule=lm.ValuationSchedule(
        effective_at=after_apple_dividend_paid.isoformat()
    )
)

valuation = aggregation_api.get_valuation(
    valuation_request=valuation_request
)

valuation_df = pd.DataFrame(valuation.data)
valuation_df

In [ ]:
generate_href(
    Demo.VALUATION, 
    after_apple_dividend_paid, 
    [[portfolio_code_fifo, portfolio_code_fifo]],
)